In [ ]:
import pandas as pd
import sqlalchemy
import os
import sys
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from datetime import date
import traceback
import ssl
engine
def sql_upload(df, name, schema, con, if_exists='append', method=None, chunksize=10, dtype=None):
    df.to_sql(
        name=name,
        con=con,
        schema=schema,
        if_exists=if_exists,
        method=method,
        index=False,
        dtype=dtype
    )
con = engine.connect()

df = pd.read_excel('https://www.cbr.ru/vfs/statistics/ms/mb_bd.xlsx', header =[2, 4], sheet_name = 0)
df.columns = ["Дата", 'Денежная база (в широком определении)', 'Наличные деньги в обращении с учетом остатков средств в кассах кредитных организаций', 
            'Корреспондентские счета кредитных организаций в Банке России', 'Обязательные резервы', 'Депозиты кредитных организаций в Банке России',  
            'Облигации Банка России у кредитных организаций', 'Обязательства Банка России по обратному выкупу ценных бумаг', 
            'Средства резервирования по валютным операциям, внесенные в Банк России']
df = df.dropna()
df['Дата'] = pd.to_datetime(df['Дата'])
df = df.drop(columns=['Облигации Банка России у кредитных организаций',
        'Обязательства Банка России по обратному выкупу ценных бумаг',
        'Средства резервирования по валютным операциям, внесенные в Банк России'])
df_1 = pd.read_excel('https://www.cbr.ru/Content/Document/File/128613/monetary_agg.xlsx', header = 0, sheet_name = 0)
df_1 = df_1.melt(id_vars=[df_1.columns[0]], var_name=df_1.columns[1], value_name = 'Млрд. руб')
df_1.columns = df_1.columns.astype(str)
df_1 = df_1.rename(columns={'Денежные агрегаты*, млрд руб.': 'Денежные агрегаты', '1993-01-01 00:00:00': 'Дата'})
df_1['Дата'] = pd.to_datetime(df_1['Дата'])
df_1 = df_1.reset_index().pivot_table(index = 'Дата', columns = 'Денежные агрегаты', values = 'Млрд. руб' )
df_1 = df_1.reset_index()
df_1 = df_1.dropna()
df_1_1 = pd.read_excel('https://www.cbr.ru/Content/Document/File/128613/monetary_agg.xlsx', header = 0, sheet_name = 1)
df_1_1 = df_1_1.melt(id_vars=[df_1_1.columns[0]], var_name=df_1_1.columns[1], value_name = 'Млрд. руб')
df_1_1.columns = df_1_1.columns.astype(str)
df_1_1
df_1_1 = df_1_1.rename(columns={'Широкая денежная масса  в разрезе секторов-держателей денег*, млрд руб.': 'Широкая денежная масса в разрезе секторов-держателей денег',
 '2015-01-01 00:00:00': 'Дата'})
df_1_1
df_1_1
df_1_1 = df_1_1.replace("01.01.2025**", "01.01.2025") 
df_1_1 = df_1_1.replace("01.10.2024**", "01.10.2024")
df_1_1
df_1_1['Дата'] = pd.to_datetime(df_1_1['Дата'])
df_1_1 = df_1_1.reset_index().pivot_table(index = 'Дата', columns = 'Широкая денежная масса в разрезе секторов-держателей денег', values = 'Млрд. руб' )
df_1_1 = df_1_1.reset_index()
df_1_1 = df_1_1.dropna()
df_2 = pd.read_excel('https://www.cbr.ru/vfs/statistics/credit_statistics/M2-M2_SA.xlsx', header = [0, 1], sheet_name = 0)
df_2.columns = ['Дата', 'Денежная масса, млрд рублей (M2)', 'Денежная масса сезонно скорректированная, млрд рублей (M2_SA)']
df_2 = df_2.dropna()
df_3 = pd.read_excel('https://www.cbr.ru/vfs/statistics/credit_statistics/broadmoney_sa.xlsx', header = [0, 1], sheet_name = 0)
df_3.columns = ['Дата', 'Широкая денежная масса, млдр рублей', 'Широкая денежная масса сезонно скорректированная, млрд рублей']
df_3 = df_3.dropna()
df_overall = df.merge(df_1, how = 'outer', on = 'Дата')
df_overall = df_overall.merge(df_1_1, how = 'outer', on = 'Дата')
df_overall = df_overall.merge(df_2, how = 'outer', on = 'Дата')
df_overall = df_overall.merge(df_3, how = 'outer', on = 'Дата')
df_overall = df_overall.drop_duplicates()
df_overall['Денежный агрегат М0'] = df_overall['Денежный агрегат М0_x'].combine_first(df_overall['Денежный агрегат М0_y'])
df_overall['Денежный агрегат М1'] = df_overall['Денежный агрегат М1_x'].combine_first(df_overall['Денежный агрегат М1_y'])
df_overall['Денежный агрегат М2'] = df_overall['Денежный агрегат М2_x'].combine_first(df_overall['Денежный агрегат М2_y'])
df_overall = df_overall.rename(columns={ 'Широкая денежная масса, млдр рублей': 'Широкая денежная масса, млрд рублей'})
df_overall['Денежный агрегат М2X (широкая денежная масса)'] = df_overall['Денежный агрегат M2X\n(широкая денежная масса)'].combine_first(df_overall['Денежный агрегат M2X (широкая денежная масса)'])
df_overall['Денежный агрегат М2X (широкая денежная масса)'] = df_overall['Денежный агрегат М2X (широкая денежная масса)'].combine_first(df_overall['Широкая денежная масса сезонно скорректированная, млрд рублей'])
df_overall['Денежный агрегат М2X (широкая денежная масса)'] = df_overall['Денежный агрегат М2X (широкая денежная масса)'].combine_first(df_overall['Широкая денежная масса, млрд рублей'])
df_overall = df_overall.drop(columns=['Денежный агрегат М0_x', 'Денежный агрегат М0_y', 'Денежный агрегат М1_x', 'Денежный агрегат М1_y', 'Денежный агрегат М2_x', 'Денежный агрегат М2_y', 'Переводные депозиты в рублях',
                                      'Переводные депозиты других финансовых организаций', 'Переводные депозиты нефинансовых организаций', 'Переводные депозиты домашних хозяйств', ' - с начислением процентов**', ' - без начисления процентов',
                                      'Другие депозиты в рублях', 'Другие депозиты других финансовых организаций', 'Другие депозиты нефинансовых организаций', 'Другие депозиты домашних хозяйств', 'Другие депозиты в иностранной валюте', 
                                      'Депозиты в иностранной валюте других финансовых организаций', 'Депозиты в иностранной валюте нефинансовых организаций', 'Депозиты в иностранной валюте домашних хозяйств', 'Депозитные и сберегательные сертификаты',
                                      'Денежный агрегат M2X\n(широкая денежная масса)', 'Денежный агрегат M2X (широкая денежная масса)', 'Широкая денежная масса, млрд рублей', 'Широкая денежная масса сезонно скорректированная, млрд рублей'])
sql_upload(df_overall, 'money_agg', 'cbr', con, dtype={'Дата':sqlalchemy.DATE})

2025-02-13 16:35:39,294 INFO sqlalchemy.engine.Engine select @@version
2025-02-13 16:35:39,296 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-02-13 16:35:39,298 INFO sqlalchemy.engine.Engine SELECT schema_name()
2025-02-13 16:35:39,299 INFO sqlalchemy.engine.Engine [generated in 0.00120s] {}
2025-02-13 16:35:39,304 INFO sqlalchemy.engine.Engine SELECT CAST('test max support' AS NVARCHAR(max))
2025-02-13 16:35:39,306 INFO sqlalchemy.engine.Engine [generated in 0.00205s] {}
2025-02-13 16:35:39,310 INFO sqlalchemy.engine.Engine SELECT 1 FROM fn_listextendedproperty(default, default, default, default, default, default, default)
2025-02-13 16:35:39,311 INFO sqlalchemy.engine.Engine [generated in 0.00141s] {}
